# 트랜스포머 모델은 활용한 번역기

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import random

import re

import matplotlib.pyplot as plt
import seaborn

plt.rc("font", family="Malgun Gothic")

### 데이터
* 데이터 : https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1

In [36]:
# 중복 제거한 데이터 로드
def clean_corpus():
    with open("../data/korean-english-park.train.ko", "r") as f:
        ko = f.read().splitlines()
    with open("../data/korean-english-park.train.en", "r") as f:
        en = f.read().splitlines()

    df = pd.DataFrame({"ko": ko, "en": en})
    cleaned_corpus = df.drop_duplicates(["ko"])

    return cleaned_corpus

In [37]:
# 전처리
def preprocess_sentence(sentence):
    sentence = sentence.lower()  # 소문자화

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)  # 기호처리
    sentence = re.sub(r'[" "]+', " ", sentence)  # 연속 공백 처리
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)  # 기타 문자 제거

    sentence = sentence.strip()  # 양쪽 공백 제거

    return sentence

In [38]:
# 토큰화
def generate_tokenizer(
    corpus, vocab_size=20000, lang="ko", pad_id=0, bos_id=1, eos_id=2, unk_id=3
):
    import sentencepiece as sp
    import os

    path = f"model/korean-english-park.train.{lang}.temp"
    tokenizer = sp.SentencePieceProcessor()

    # 처음 학습한 sp 모델이 있는 지 확인
    if not os.path.exists(f"{lang}_spm.model"):
        # 토큰화 학습에 필요한 파일 생성
        with open(path, "w") as f:
            for row in corpus:
                f.write(str(row) + "\n")

        # 토큰화 모델 학습
        sp.SentencePieceTrainer.Train(
            f"--input={path} --model_prefix={lang}_spm --vocab_size={vocab_size} --pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}"
        )
    tokenizer.Load(f"{lang}_spm.model")

    return tokenizer

In [39]:
# 전처리한 코퍼스 생성
cleaned_corpus = clean_corpus()

cleaned_corpus["ko"] = cleaned_corpus["ko"].apply(lambda x: preprocess_sentence(x))
cleaned_corpus["en"] = cleaned_corpus["en"].apply(lambda x: preprocess_sentence(x))

In [40]:
# 토크나이저 생성
ko_tokenizer = generate_tokenizer(cleaned_corpus["ko"])
en_tokenizer = generate_tokenizer(cleaned_corpus["en"], lang="en")
en_tokenizer.set_encode_extra_options("bos:eos")

True

In [41]:
# 토크나이징
cleaned_corpus.loc[:, "ko"] = cleaned_corpus["ko"].apply(
    lambda x: ko_tokenizer.EncodeAsIds(x)
)
cleaned_corpus.loc[:, "en"] = cleaned_corpus["en"].apply(
    lambda x: en_tokenizer.EncodeAsIds(x)
)

In [43]:
# 데이터 길이 50이하만 유지
cleaned_corpus_short = cleaned_corpus[cleaned_corpus["ko"].apply(len) <= 50]

In [45]:
cleaned_corpus_short

,ko,en
0,"[1071, 358, 522, 480, 7, 1285, 1879, 8, 1204, ...","[1, 354, 11, 1466, 8806, 14, 23, 59, 115, 128,..."
1,"[172, 3249, 390, 3710, 27, 3475, 186, 19, 3249...","[1, 186, 10, 2899, 10, 431, 564, 348, 59, 10, ..."
2,"[69, 1204, 8, 144, 11443, 28, 5495, 438, 463, 4]","[1, 144, 102, 13430, 190, 2925, 6, 56, 33, 77,..."
3,"[4, 139, 42, 19, 5739, 3652, 3249, 390, 3710, ...","[1, 323, 7, 1697, 16, 91, 14046, 302, 6652, 7,..."
7,"[19, 465, 189, 12, 823, 71, 12, 796, 1135, 16,...","[1, 1804, 936, 6, 33, 115, 27, 154, 8, 12, 150..."
...,...,...
94118,"[103, 6, 46, 20, 9, 19582, 762, 25, 22, 7, 501...","[1, 104, 40, 1768, 14, 9, 6096, 1371, 864, 9, ..."
94119,"[1562, 5419, 12937, 61, 8, 19009, 25, 12511, 2...","[1, 20, 5230, 2885, 6, 101, 4344, 8, 195, 7, 3..."
94120,"[69, 15632, 17, 1019, 1235, 18, 6496, 1831, 14...","[1, 56, 4, 1440, 11, 1371, 864, 23, 539, 14, 3..."
94121,"[19009, 25, 22, 7, 896, 8, 4053, 3927, 1283, 4...","[1, 18, 6256, 13, 39, 975, 372, 3573, 4, 1385,..."


In [46]:
# 패딩처리
enc_train = tf.keras.preprocessing.sequence.pad_sequences(
    cleaned_corpus_short["ko"],
    padding="post",
)


dec_train = tf.keras.preprocessing.sequence.pad_sequences(
    cleaned_corpus_short["en"],
    padding="post",
)

In [47]:
# 영어는 토큰수가 한국어 보다 많은데 이를 50까지만 사용
# 괜찮은건지 실험할 예정
enc_train.shape, dec_train.shape

((75579, 50), (75579, 129))

In [49]:
enc_train = tf.keras.preprocessing.sequence.pad_sequences(
    cleaned_corpus_short["ko"],
    maxlen=dec_train.shape[1],
    padding="post",
)

In [50]:
enc_train.shape, dec_train.shape

((75579, 129), (75579, 129))

## 모델

In [52]:
# 포지셔널 인코딩
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    return sinusoid_table

In [53]:
# 멀티헤드 어텐션
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)  # Linear Layer
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)  # 나누기 위하여 타입 변경

        scaled_qk = tf.matmul(Q, K, transpose_b=True)  # q, k의 유사도 측정
        scaled_qk /= tf.sqrt(d_k)  # 스케일 조정

        # 마스크에 해당하는 아주 큰 값을 빼서 softmax를 통과하면 0에 수렴하게 만듦
        if mask is not None:
            scaled_qk += mask * -1e9

        attentions = tf.nn.softmax(scaled_qk, -1)
        out = tf.matmul(attentions, V)  # 어텐션 가중치 반영

        return out, attentions

    def split_heads(self, x):
        """
        Embedding을 Head의 수로 분할하는 함수

        x: [ batch x length x emb ]
        return: [ batch x heads x length x self.depth ]
        """

        split_x = tf.reshape(x, (x.shape[0], -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, [0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        """
        분할된 Embedding을 하나로 결합하는 함수

        x: [ batch x heads x length x self.depth ]
        return: [ batch x length x emb ]
        """
        # split_heads의 역순 진행
        combined_x = tf.transpose(x, [0, 2, 1, 3])
        combined_x = tf.reshape(
            combined_x, (combined_x.shape[0], combined_x.shape[1], self.d_model)
        )

        return combined_x

    def call(self, Q, K, V, mask):
        # 리니어 적용
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        # 머리 쪼개기
        WQ_split = self.split_heads(WQ)
        WK_split = self.split_heads(WK)
        WV_split = self.split_heads(WV)

        # 어텐션
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_split, WK_split, WV_split, mask
        )

        # 머리 합치기
        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [54]:
# FFN
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation="relu")
        self.w_2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)

        return out

### 마스크

In [55]:
def generate_padding_mask(seq):
    # 실수가 아닌 것을 전부 0으로
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]


def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)


def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [56]:
# 마스킹 확인
# batch, length = 16, 20
# src_padding = 5
# tgt_padding = 15

# src_pad = tf.zeros(shape=(batch, src_padding))
# tgt_pad = tf.zeros(shape=(batch, tgt_padding))

# sample_data = tf.ones(shape=(batch, length))

# sample_src = tf.concat([sample_data, src_pad], axis=-1)
# sample_tgt = tf.concat([sample_data, tgt_pad], axis=-1)

# enc_mask, dec_enc_mask, dec_mask = generate_masks(sample_src, sample_tgt)

# fig = plt.figure(figsize=(7, 7))

# ax1 = fig.add_subplot(131)
# ax2 = fig.add_subplot(132)
# ax3 = fig.add_subplot(133)

# ax1.set_title("1) Encoder Mask")
# ax2.set_title("2) Encoder-Decoder Mask")
# ax3.set_title("3) Decoder Mask")

# ax1.imshow(enc_mask[:3, 0, 0].numpy(), cmap="Dark2")
# ax2.imshow(dec_enc_mask[0, 0].numpy(), cmap="Dark2")
# ax3.imshow(dec_mask[0, 0].numpy(), cmap="Dark2")

# plt.show()

### 인코더 디코더

In [57]:
# 인코더층
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        # Multi-Head Attention
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.dropout(out)
        out += residual

        # Position-Wise Feed Forward Networn
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, enc_attn

In [58]:
# 디코더층
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, n_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, mask, casual_mask):

        # Multi-Head Attention
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, mask)
        out = self.dropout(out)
        out += residual

        # encoder-decoder Attention
        residual = out
        out = self.norm_2(x)
        out, enc_dec_attn = self.enc_dec_attn(out, enc_out, enc_out, casual_mask)
        out = self.dropout(out)
        out += residual

        # Position-Wise Feed Forward Network
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, dec_attn, enc_dec_attn

In [59]:
# 전체 인코더
class Encoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [
            EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)
        ]

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

In [60]:
# 전체 디코더
class Decoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [
            DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)
        ]

    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = self.dec_layers[i](
                out, enc_out, causality_mask, padding_mask
            )

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

### 트랜스포머

In [61]:
class Transformer(tf.keras.Model):
    def __init__(
        self,
        n_layers,
        d_model,
        n_heads,
        d_ff,
        src_vocab_size,
        tgt_vocab_size,
        pos_len,
        dropout=0.2,
        shared=True,
    ):
        super(Transformer, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)

        self.enc_embedding = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.dec_embedding = tf.keras.layers.Embedding(tgt_vocab_size, d_model)
        self.positionl = positional_encoding(pos_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared = shared
        if self.shared:
            self.fc.set_weights(tf.transpose(self.dec_embedding.weights))

    def embedding(self, emb, x):
        """
        입력된 정수 배열을 Embedding + Pos Encoding
        + Shared일 경우 Scaling 작업 포함

        x: [ batch x length ]
        return: [ batch x length x emb ]
        """

        out = emb(x)

        if self.shared:
            out *= tf.sqrt(self.d_model)

        # 포지셔널 인코딩은 batch차원이 없어서 확장 : [tf.newaxis, ...]
        # 데이터 길이 만큼만 더하기 : [:, :x.shape[1], :]
        pos_encoding = tf.constant(self.positionl, dtype=tf.float32)
        out += pos_encoding[tf.newaxis, ...][:, : x.shape[1], :]
        out = self.dropout(out)

        return out

    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        # 임베딩
        enc_in = self.embedding(self.enc_embedding, enc_in)
        dec_in = self.embedding(self.dec_embedding, dec_in)

        # 인코더
        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        # 디코더
        dec_out, dec_attns, enc_dec_attns = self.decoder(
            dec_in, enc_out, dec_mask, causality_mask
        )
        # 최종 출력
        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, enc_dec_attns

### 평가 및 시각화

In [62]:
# Attention 시각화 함수
def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
    def draw(data, ax, x="auto", y="auto"):
        import seaborn

        seaborn.heatmap(
            data,
            square=True,
            vmin=0.0,
            vmax=1.0,
            cbar=False,
            ax=ax,
            xticklabels=x,
            yticklabels=y,
        )

    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Encoder Layer", layer + 1)
        for h in range(4):
            draw(enc_attns[layer][0, h, : len(src), : len(src)], axs[h], src, src)
        plt.show()

    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Decoder Self Layer", layer + 1)
        for h in range(4):
            draw(dec_attns[layer][0, h, : len(tgt), : len(tgt)], axs[h], tgt, tgt)
        plt.show()

        print("Decoder Src Layer", layer + 1)
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        for h in range(4):
            draw(dec_enc_attns[layer][0, h, : len(tgt), : len(src)], axs[h], src, tgt)
        plt.show()

In [63]:
# 번역 생성 함수


def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences(
        [tokens], maxlen=enc_train.shape[-1], padding="post"
    )

    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = generate_masks(
            _input, output
        )

        predictions, enc_attns, dec_attns, dec_enc_attns = model(
            _input, output, enc_padding_mask, combined_mask, dec_padding_mask
        )

        predicted_id = (
            tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
        )

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [64]:
# 번역 생성 및 Attention 시각화 결합
def translate(sentence, model, src_tokenizer, tgt_tokenizer, plot_attention=False):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = evaluate(
        sentence, model, src_tokenizer, tgt_tokenizer
    )

    print("Input: %s" % (sentence))
    print("Predicted translation: {}".format(result))

    if plot_attention:
        visualize_attention(pieces, result.split(), enc_attns, dec_attns, dec_enc_attns)

### 훈련

In [65]:
# 학습률, 옵티마이저
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step**-0.5
        arg2 = step * (self.warmup_steps**-1.5)

        return (self.d_model**-0.5) * tf.math.minimum(arg1, arg2)

In [66]:
# loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction="none"
)


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    # Masking 되지 않은 입력의 개수로 Scaling
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

In [67]:
# Train Step


@tf.function()
def train_step(src, tgt, model, optimizer):
    gold = tgt[:, 1:]

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

    # 계산된 loss에 tf.GradientTape()를 적용해 학습을 진행합니다.
    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = model(
            src, tgt, enc_mask, dec_enc_mask, dec_mask
        )
        loss = loss_function(gold, predictions[:, :-1])

    # 최종적으로 optimizer.apply_gradients()가 사용됩니다.
    variables = model.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [68]:
from tqdm import tqdm  # tqdm
import random

EPOCHS = 20
BATCH_SIZE = 64

# 모델 설정
n_layers = 2
d_model = 256
n_heads = 8
d_ff = 256
src_vocab_size = ko_tokenizer.vocab_size()
tgt_vocab_size = en_tokenizer.vocab_size()
pos_len = enc_train.shape[1]

transformer = Transformer(
    n_layers, d_model, n_heads, d_ff, src_vocab_size, tgt_vocab_size, pos_len
)

learning_rate = LearningRateScheduler(512)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)

example_sentence = [
    "오바마는 대통령이다.",
    "시민들은 도시 속에 산다.",
    "커피는 필요 없다.",
    "일곱 명의 사망자가 발생했다.",
]

transformer = Transformer(
    n_layers, d_model, n_heads, d_ff, src_vocab_size, tgt_vocab_size, pos_len
)

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)  # tqdm

    for batch, idx in enumerate(t):
        loss, enc_attns, dec_attns, dec_enc_attns = train_step(
            enc_train[idx : idx + BATCH_SIZE],
            dec_train[idx : idx + BATCH_SIZE],
            transformer,
            optimizer,
        )

        total_loss += loss

        t.set_description_str("Epoch %2d" % (epoch + 1))  # tqdm
        t.set_postfix_str("Loss %.4f" % (total_loss.numpy() / (batch + 1)))  # tqdm

    for s in example_sentence:
        translate(s, transformer, ko_tokenizer, en_tokenizer)

Epoch  1: 100%|██████████| 1181/1181 [01:41<00:00, 11.59it/s, Loss 6.1552]


Input: 오바마는 대통령이다.
Predicted translation: obama s obama s obama s obama s obama s obama s obama s obama s obama se .
Input: 시민들은 도시 속에 산다.
Predicted translation: the first time , the first time .
Input: 커피는 필요 없다.
Predicted translation: the dow is not .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: the wounded were killed to the attack .


Epoch  2: 100%|██████████| 1181/1181 [01:35<00:00, 12.31it/s, Loss 4.6437]


Input: 오바마는 대통령이다.
Predicted translation: obama has been a campaign .
Input: 시민들은 도시 속에 산다.
Predicted translation: the biggest is across the world .
Input: 커피는 필요 없다.
Predicted translation: i think you can do not .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: the death toll was reported .


Epoch  3: 100%|██████████| 1181/1181 [01:36<00:00, 12.26it/s, Loss 4.1201]


...

Input: 오바마는 대통령이다.
Predicted translation: obama would oppose it .
Input: 시민들은 도시 속에 산다.
Predicted translation: the protest is sitting in the mountain .
Input: 커피는 필요 없다.
Predicted translation: drink coffee is not prepared to speech .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: six other people were dead tuesday


Epoch 19: 100%|██████████| 1181/1181 [01:37<00:00, 12.13it/s, Loss 1.0482]


Input: 오바마는 대통령이다.
Predicted translation: obama is the most important thing for him .
Input: 시민들은 도시 속에 산다.
Predicted translation: protesting cities s calm in .
Input: 커피는 필요 없다.
Predicted translation: drink coffee is not prepared to speak in .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: tuesday s death toll could , which has been deaths in several days .


Epoch 20: 100%|██████████| 1181/1181 [01:38<00:00, 12.04it/s, Loss 0.9778]


Input: 오바마는 대통령이다.
Predicted translation: obama is the president elect he s going to be president elect barack obama .
Input: 시민들은 도시 속에 산다.
Predicted translation: the protest in cities is staying .
Input: 커피는 필요 없다.
Predicted translation: don t need to speech don t care .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: the nine year old eleven death saturday was dead .


### 실험 결과 비교
*epoch 별 번역 기록을 txt로 저장하여 보관*

#### 1. 하이퍼 파라미터 조정 이전에, 토큰화 패딩을 무엇을 기준으로 할지를 먼저 비교. 

 lms에서 한국어 토큰 수 50개 이하를 사용하는데, 영어의 토큰수는 이보다 많았고 어디에 맞추는 것이 좋을 지 비교.  
  일단 실험 전에는 긴 쪽으로, 영어에 맞추는게 성능이 좋을 것이라고 생각됨. 한국어와 영어가 어순이 동일하지도 않고, 영어를 중간까지 짜른것만으로는 번역의 학습이 잘 안 될것으로 예상  
  
  ##### 1.1 한국어에 맞추었을 때  
  sequence length : 50   
  epoch 당 시간 : 40초대
  ```text
  poch 20: 100%|██████████| 1181/1181 [00:42<00:00, 27.56it/s, Loss 0.9116]
    Input: 오바마는 대통령이다.
    Predicted translation: obama is as obama .
    Input: 시민들은 도시 속에 산다.
    Predicted translation: the city is to get to the city in the city just living , street .
    Input: 커피는 필요 없다.
    Predicted translation: no one does notapal .
    Input: 일곱 명의 사망자가 발생했다.
    Predicted translation: seven deaths were died .
  ```
  ##### 1.2 영어에 맞추었을 때   
  sequence length : 129  
  epoch 당 시간 : 1분 30초대(2배)  

  ```text
  Epoch 15: 100%|██████████| 1181/1181 [01:38<00:00, 11.98it/s, Loss 1.4021]
    Input: 오바마는 대통령이다.
    Predicted translation: obama is the president elected .
    Input: 시민들은 도시 속에 산다.
    Predicted translation: the city has been sitting in place .
    Input: 커피는 필요 없다.
    Predicted translation: drink coffee is not needing .
    Input: 일곱 명의 사망자가 발생했다.
    Predicted translation: on tuesday , the seventh was dead .
  ```

  ##### 1.3 결론
  20개의 결과를 보고 가장 좋았던 것을 기록하였는데, 1.2처럼 영어로 기준을 하는게 더 좋은 결과를 보이는 듯 하다. 또한 1.1의 경우 금방 비슷한 결과만 나오게 되었다. 
  

#### 2. 파라미터를 조정
 